In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import inv
import math

In [2]:
# Two telecommunication companies, two tech companies, and Walmart
expected_return = []
tickers = ['CMCSA','T', 'AAPL', 'MSFT', 'WMT']


start_date = '2014-01-01'
end_date = '2014-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return = ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2015-01-01'
end_date = '2015-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2016-01-01'
end_date = '2016-12-30'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
    
start_date = '2017-01-01'
end_date = '2017-12-29'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2018-01-01'
end_date = '2018-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)

Here expected return for each stock is calculated throughout years 2014 to 2018. To see results, the last line in the following cell may be uncommented.

In [3]:
expected_return = np.array(expected_return)
years = [];
for i in range(2014, 2019):
    years.append(i)

returns_df = pd.DataFrame(expected_return.reshape(5,5), columns = tickers)
returns_df['year'] = years
#display(returns_df)

,CMCSA,T,AAPL,MSFT,WMT,year
0,1.705207,-28.388810,27.456902,11.344808,-3.731348,2014
1,-10.146865,-18.620589,-11.904417,9.353252,-35.554956,2015
2,18.108101,4.171407,7.569124,8.095585,6.627371,2016
3,9.813918,-20.169875,41.511218,31.603032,36.547809,2017
4,-18.278692,-32.165617,-8.887455,15.891587,-8.055473,2018


In the following cell we create a list of stocks and compare each one to their data from 2018 to 2014. The data files are read in and stored in a list which is used in the first outer loop to get the significant features for comparison. The outer loop iterates through each year. The inner loop takes each stock from the selected year and adds it to its own list. It then takes the appropriate Expected Return values (calculated above) and appends them to each stock as a new column. Lastly, we convert these lists to dataframes and store the dataframes in another list titled datatables. To see results, the last two lines in the following cell may be uncommented.

In [19]:
# Read in all datasets
df2018 = pd.read_csv("2018_Financial_Data.csv")
df2017 = pd.read_csv("2017_Financial_Data.csv")
df2016 = pd.read_csv("2016_Financial_Data.csv")
df2015 = pd.read_csv("2015_Financial_Data.csv")
df2014 = pd.read_csv("2014_Financial_Data.csv")

# Insert data into list for iteration and initialize other lists
datatables = []
listtotal = []
list_stocks = []
list_years = [df2018, df2017, df2016, df2015, df2014]
for i in range(len(list_years)):
    
    # Clear list_stocks for the upcoming year
    if len(list_stocks)!=0:
        list_stocks.clear()
        
    # Modify data to get desired amount of features
    financial_data = pd.DataFrame(list_years[i], columns = ['Symbol', 'Revenue', 'Revenue Growth', 'Gross Profit', 
                                                  'Operating Income', 'Earnings before Tax', 'Free Cash Flow', 
                                                  'Net Income', 'Total current assets',
                                                  'Operating Expenses',  'Net Debt', 'Short-term debt', 'Long-term debt', 
                                                  'Total shareholders equity', 'Weighted Average Shs Out', 
                                                  'Total current liabilities', 'Total debt', 'Total liabilities']) 
    
    for j in range(len(tickers)):
        list_stocks.append(pd.DataFrame(financial_data.loc[financial_data['Symbol'] == tickers[j], :]))
        
        if i == 0:
            # Add stocks to their own list
            listtotal.append(list_stocks[j])
            
        elif i == (len(list_years)-1):

            listtotal[j] = listtotal[j].append(list_stocks[j])
            # Add on Expected Return column to each stock
            listtotal[j]['Expected Return'] = [expected_return[j],expected_return[j+len(tickers)], 
                                            expected_return[j+(2*len(tickers))], expected_return[j+(3*len(tickers))],
                                            expected_return[j+(4*len(tickers))]]
            datatables.append(pd.DataFrame(listtotal[j]))
            
        else:
            listtotal[j] = listtotal[j].append(list_stocks[j])
            
#for k in range(0, len(datatables)):
    #display(datatables[k])

,Symbol,Revenue,Revenue Growth,Gross Profit,Operating Income,Earnings before Tax,Free Cash Flow,Net Income,Total current assets,Operating Expenses,Net Debt,Short-term debt,Long-term debt,Total shareholders equity,Weighted Average Shs Out,Total current liabilities,Total debt,Total liabilities,Expected Return
0,CMCSA,9.450700e+10,0.1115,9.450700e+10,1.900900e+10,1.511100e+10,1.198500e+10,1.173100e+10,2.184800e+10,7.549800e+10,1.080000e+11,4.398000e+09,1.070000e+11,7.161300e+10,4.549505e+09,2.760300e+10,1.120000e+11,1.790000e+11,1.705207
1199,CMCSA,8.502900e+10,0.0532,8.502900e+10,1.801800e+10,1.516600e+10,9.617000e+09,2.273500e+10,1.634300e+10,6.701100e+10,6.112800e+10,5.134000e+09,5.942200e+10,6.861600e+10,4.673772e+09,2.199300e+10,6.455600e+10,1.180030e+11,-10.146865
1157,CMCSA,8.073600e+10,0.0836,8.073600e+10,1.683100e+10,1.397600e+10,8.554000e+09,8.678000e+09,1.636100e+10,6.390500e+10,5.597300e+10,5.480000e+09,5.556600e+10,5.394300e+10,4.785665e+09,2.153500e+10,6.104600e+10,1.243260e+11,18.108101
1068,CMCSA,7.451000e+10,0.0834,7.451000e+10,1.599800e+10,1.312200e+10,9.438000e+09,8.163000e+09,1.230300e+10,5.851200e+10,5.030500e+10,3.627000e+09,4.899400e+10,5.226900e+10,4.914459e+09,1.817800e+10,5.262100e+10,1.125960e+11,9.813918
989,CMCSA,6.877500e+10,0.0637,6.877500e+10,1.490400e+10,1.225300e+10,8.360000e+09,8.380000e+09,1.353100e+10,5.387100e+10,4.356900e+10,4.217000e+09,4.386400e+10,5.271100e+10,5.152595e+09,1.741000e+10,4.808100e+10,1.061180e+11,-18.278692


,Symbol,Revenue,Revenue Growth,Gross Profit,Operating Income,Earnings before Tax,Free Cash Flow,Net Income,Total current assets,Operating Expenses,Net Debt,Short-term debt,Long-term debt,Total shareholders equity,Weighted Average Shs Out,Total current liabilities,Total debt,Total liabilities,Expected Return
18,T,1.710000e+11,0.0636,9.133700e+10,2.609600e+10,2.429000e+10,2.284400e+10,1.937000e+10,5.142700e+10,6.524100e+10,1.710000e+11,1.025500e+10,1.660000e+11,1.840000e+11,7.278000e+09,6.442000e+10,1.770000e+11,3.380000e+11,-28.388810
2578,T,1.605460e+11,-0.0198,8.273600e+10,1.997000e+10,1.474200e+10,1.736300e+10,2.945000e+10,7.914600e+10,6.276600e+10,1.138480e+11,3.837400e+10,1.259720e+11,1.408610e+11,6.139000e+09,8.138900e+10,1.643460e+11,3.020900e+11,-18.620589
2490,T,1.637860e+11,0.1157,8.659600e+10,2.354300e+10,1.945500e+10,1.692600e+10,1.297600e+10,3.836900e+10,6.305300e+10,1.177250e+11,9.832000e+09,1.136810e+11,1.231350e+11,6.141000e+09,5.057600e+10,1.235130e+11,2.797110e+11,4.171407
2319,T,1.468010e+11,0.1084,7.975500e+10,2.478500e+10,2.035000e+10,1.666200e+10,1.334500e+10,3.599200e+10,5.497000e+10,1.210300e+11,7.636000e+09,1.185150e+11,1.226710e+11,6.152000e+09,4.781600e+10,1.261510e+11,2.790320e+11,-20.169875
2171,T,1.324470e+11,0.0287,7.230200e+10,1.221200e+10,1.006100e+10,1.013900e+10,6.442000e+09,3.360600e+10,6.009000e+10,7.323100e+10,6.056000e+09,7.577800e+10,8.971600e+10,5.187000e+09,3.728200e+10,8.183400e+10,2.065640e+11,-32.165617


,Symbol,Revenue,Revenue Growth,Gross Profit,Operating Income,Earnings before Tax,Free Cash Flow,Net Income,Total current assets,Operating Expenses,Net Debt,Short-term debt,Long-term debt,Total shareholders equity,Weighted Average Shs Out,Total current liabilities,Total debt,Total liabilities,Expected Return
6,AAPL,2.660000e+11,0.1586,1.020000e+11,7.089800e+10,7.290300e+10,6.412100e+10,5.953100e+10,1.310000e+11,3.094100e+10,4.818200e+10,2.074800e+10,9.373500e+10,1.070000e+11,4.829926e+09,1.160000e+11,1.140000e+11,2.590000e+11,27.456902
4297,AAPL,2.292340e+11,0.0630,8.818600e+10,6.134400e+10,6.408900e+10,5.177400e+10,4.835100e+10,1.286450e+11,2.684200e+10,4.149900e+10,1.847300e+10,9.720700e+10,1.340470e+11,5.165228e+09,1.008140e+11,1.156800e+11,2.412720e+11,-11.904417
4156,AAPL,2.156390e+11,-0.0773,8.426300e+10,6.002400e+10,6.137200e+10,5.349700e+10,4.568700e+10,1.068690e+11,2.423900e+10,1.987700e+10,1.160500e+10,7.542700e+10,1.282490e+11,5.388443e+09,7.900600e+10,8.703200e+10,1.934370e+11,7.569124
3516,AAPL,2.337150e+11,0.2786,9.362600e+10,7.123000e+10,7.251500e+10,6.977800e+10,5.339400e+10,8.937800e+10,2.239600e+10,2.272700e+10,1.099900e+10,5.332900e+10,1.193550e+11,5.702722e+09,8.061000e+10,6.432800e+10,1.709900e+11,41.511218
3234,AAPL,1.827950e+11,0.0695,7.053700e+10,5.250300e+10,5.348300e+10,4.990000e+10,3.951000e+10,6.853100e+10,1.803400e+10,1.021800e+10,6.308000e+09,2.898700e+10,1.115470e+11,5.987867e+09,6.344800e+10,3.529500e+10,1.202920e+11,-8.887455


,Symbol,Revenue,Revenue Growth,Gross Profit,Operating Income,Earnings before Tax,Free Cash Flow,Net Income,Total current assets,Operating Expenses,Net Debt,Short-term debt,Long-term debt,Total shareholders equity,Weighted Average Shs Out,Total current liabilities,Total debt,Total liabilities,Expected Return
7,MSFT,1.100000e+11,0.1428,7.200700e+10,3.505800e+10,3.647400e+10,3.225200e+10,1.657100e+10,1.700000e+11,3.694900e+10,-5.196000e+10,3.998000e+09,7.781000e+10,8.271800e+10,7.683198e+09,5.848800e+10,8.180800e+10,1.760000e+11,11.344808
4298,MSFT,9.657100e+10,0.0594,6.231000e+10,2.902500e+10,2.990100e+10,3.137800e+10,2.548900e+10,1.626960e+11,3.328500e+10,-4.141500e+10,1.012100e+10,8.144500e+10,8.771100e+10,7.720515e+09,5.574500e+10,9.156600e+10,1.626010e+11,9.353252
4157,MSFT,9.115400e+10,-0.0259,5.837400e+10,2.607800e+10,2.563900e+10,2.498200e+10,2.053900e+10,1.396600e+11,3.229600e+10,-5.977900e+10,1.290400e+10,4.055700e+10,7.199700e+10,7.860467e+09,5.935700e+10,5.346100e+10,1.214710e+11,8.095585
3517,MSFT,9.358000e+10,0.0777,6.054200e+10,1.816100e+10,1.850700e+10,2.372400e+10,1.219300e+10,1.227970e+11,4.238100e+10,-6.123400e+10,7.484000e+09,2.780800e+10,8.008300e+10,8.089575e+09,4.964700e+10,3.529200e+10,9.438900e+10,31.603032
3235,MSFT,8.683300e+10,0.1154,5.975500e+10,2.775900e+10,2.782000e+10,2.701700e+10,2.207400e+10,1.142460e+11,3.199600e+10,-6.306400e+10,2.000000e+09,2.064500e+10,8.978400e+10,8.260412e+09,4.562500e+10,2.264500e+10,8.260000e+10,15.891587


,Symbol,Revenue,Revenue Growth,Gross Profit,Operating Income,Earnings before Tax,Free Cash Flow,Net Income,Total current assets,Operating Expenses,Net Debt,Short-term debt,Long-term debt,Total shareholders equity,Weighted Average Shs Out,Total current liabilities,Total debt,Total liabilities,Expected Return
90,WMT,5.000000e+11,0.0298,1.270000e+11,2.043700e+10,1.446200e+10,1.866400e+10,9.862000e+09,5.966400e+10,1.070000e+11,3.973100e+10,9.662000e+09,3.682500e+10,7.786900e+10,2.962381e+09,7.852100e+10,4.648700e+10,1.240000e+11,-3.731348
7,WMT,4.858730e+11,0.0078,1.246170e+11,2.276400e+10,1.984700e+10,2.151000e+10,1.364300e+10,5.768900e+10,1.018530e+11,3.907100e+10,3.920000e+09,4.201800e+10,7.779800e+10,3.073190e+09,6.692800e+10,4.593800e+10,1.182900e+11,-35.554956
7,WMT,4.821300e+11,-0.0073,1.211460e+11,2.410500e+10,2.125200e+10,1.671000e+10,1.469400e+10,6.023900e+10,9.704100e+10,4.132900e+10,6.004000e+09,4.403000e+10,8.054600e+10,3.201893e+09,6.461900e+10,5.003400e+10,1.159700e+11,6.627371
7,WMT,4.856510e+11,0.0196,1.205650e+11,2.714700e+10,2.434800e+10,1.696000e+10,1.636300e+10,6.327800e+10,9.341800e+10,4.103000e+10,6.670000e+09,4.349500e+10,8.139400e+10,3.223190e+09,6.525300e+10,5.016500e+10,1.175530e+11,36.547809
7,WMT,4.762940e+11,0.0163,1.182250e+11,2.687200e+10,2.412700e+10,1.086900e+10,1.602200e+10,6.118500e+10,9.135300e+10,4.936000e+10,1.208200e+10,4.455900e+10,7.625500e+10,3.235772e+09,6.934500e+10,5.664100e+10,1.234120e+11,-8.055473


In [20]:
# Create an array of the tables above
#dataTables = [s1_total, s2_total, s3_total, s4_total, s5_total]

# Loop through each of the tables and calculate the pararmeters for multiple linear regrression 
# After calculating the parameters, test the regression and calcualte RMSE
for i in range(0, len(datatables)):
    
    # Find what are the X and Y matrices for normal equation
    x = datatables[i].drop('Symbol',1)
    x = x.drop('Expected Return', 1).to_numpy()
    y = datatables[i]['Expected Return'].to_numpy()
    
    # Calculate the parameters
    XTX = x.transpose().dot(x)
    XTX_inverse = inv(np.matrix(XTX))
    XTX_inverse_X = XTX_inverse.dot(x.transpose())
    param_theta = XTX_inverse_X.dot(y)
    

    # Find RMSE for the model using the training data and targets we have
    error_sum = 0
    for j in range (0,5):
        y_pred = param_theta.dot(x[j])
        #print(y_pred)
        pred_difference = y_pred - returns_df[tickers[i]][j]
        error_sum = error_sum + ((pred_difference**2) / 5)
    RMSE = math.sqrt(error_sum)
    print("RMSE for ", tickers[i], " is ", RMSE) #print the RMSE

RMSE for  CMCSA  is  2.3027827207312885
RMSE for  T  is  105.51891347566765
RMSE for  AAPL  is  129.46772053770098
RMSE for  MSFT  is  437.5444310631503
RMSE for  WMT  is  15.728002433485424
